# Efficient Spam Email Detection





## Introduction
Spam emails can sometimes be annoying and hard to detect when they are nested in mailbox. We have to read them one by one to decide whether we want to remove it or not. Generally it takes a great amount of time to sanitize before the new ones arrive. It would be helpful if we can construct a classifier to detect the spam emails based on our previous effort on removing the spams. In this project, we are going to build and efficiently train a linear SVM spam email classifier from scratch with gradient descent algorithm. The final goal of this project is to make sure that our classifer can acheive at least 80% accuracy on detecting the new spam emails. 

## Installation
Before getting started, you'll need to install various packages that will be used in this project. It is strongly recommanded to use Anaconda since all the open source packages can be individually installed from the Anaconda repository. By calling

$ pip install [packages you want to install]

Or

$ conda install [packages you want to install]

Anaconda compiles and builds all the packages in the Anaconda repository itself. The packages required are listed as follow.

You can also refer to requirements.txt to figure out the required packages.

## Text Processing
The emails have already been labeled as 1 if they are ham (not spam), and -1 if they are spam. The lines of the emails have already been slightly processed, such that different words are space delimited, however little other processing has occurred. The data is distributed by course instructor for educational purpose.

In [31]:
import gzip
import math
import numpy as np
import scipy.sparse as sp
import scipy.optimize
from collections import Counter

In [32]:
# Helper function to load the data
def load_data(emails_filepath, labels_filepath):
    with gzip.open(emails_filepath, 'rt') as f:
        emails = f.readlines()
    with gzip.open(labels_filepath, 'rt') as f:
        labels = np.loadtxt(f)
    return emails, labels

A wrapper function to cache the loaded dataset. This helps it not take quite as long to re-run. 

In [33]:
global DATA_CACHE
DATA_CACHE = None

# Load the data
def get_data():
    global DATA_CACHE
    if DATA_CACHE is None:
        DATA_CACHE = load_data('X1.txt.gz', 'y1.txt.gz')
    return DATA_CACHE

Construct a sparse tf-idf matrix where the length of each row represents the vocabulary of the dataset and the number of rows is equal to the number of emails in the dataset. We have 10,000 datapoints but more than 200,000 features for each email, however, most of the emails don't have a specific word. It would be efficient to convert it into sparse format where 0 indicates the word is not contained in this email and 1 vice versa. 

In [34]:
# Construct a sparse tf-idf feature matrix
def tfidf(docs):
    all_words = set([a for a in " ".join(docs).split(" ") if a != ""])
    all_words_dict = {k:i for i,k in enumerate(all_words)}
    word_counts = [Counter([a for a in d.split(" ") if a != ""]) for d in docs]

    data = [a for wc in word_counts for a in wc.values()]
    rows = [i for i,wc in enumerate(word_counts) for a in wc.values()]
    cols = [all_words_dict[k] for wc in word_counts for k in wc.keys()]
    # Convert it into sparse
    X = sp.coo_matrix((data, (rows,cols)), (len(docs), len(all_words)))

    idf = np.log(float(len(docs))/np.asarray((X > 0).sum(axis=0))[0])
    return X * sp.diags(idf), list(all_words)

## SVM classification
We are going to build a linear SVM classifier in this section. In general, all the machine learning problem can be roughly divided into the following three sections in terms of mathematic interpretation.

1. Hypothesis function: A hypothesis funtion is a mapping from the input space to the prediction space.
2. Loss function: A mapping from predictions and true outputs to real numbers, is a measure of how good a prediction is. In our case, SVM is using hinge loss.
3. Optimization: Minimize the loss function using different algorithms (in our case we use gradient descent)

The format of objective function (also known as loss function) to implement is:$$
\mathcal L(y, X, \theta, \lambda) = \text{sum}\left\{\text{max}\{1 - y\cdot \theta^T X, 0 \}\right\} + \frac\lambda2(\theta\cdot\theta)
$$ where $X$ is the input, $y$ is the true label, $\theta$ is the model weight, and $\lambda$ is the regularization weight. The inner `max` operation is performed over each element, and the outer `sum` operation returns the sum of the elements of the vector.

The gradient is:$$
    v = \mathbb1\left\{y* (X \theta)) <= 1\right\}
\\    
\\    \frac{d\theta}{d\mathcal L} = \text{sum}_\text{col}\{-y* v*X\} + \lambda\theta
$$
where the $\text{sum}_\text{col}$ operation calculates the sum over each column, producing a vector the same shape as $\theta$.



In [35]:
class SVM:
    
    # Initialize the SVM attributes and initialize the weights vector to the zero vector. 
    def __init__(self, X, y, reg):
        self.X = X
        self.y = y
        self.reg = reg
        self.theta = np.zeros(X.shape[1])
    
    # Calculate the objective value of the SVM。
    def objective(self, X, y):
        return np.maximum(np.ones(self.X.shape[0]) - self.theta @ self.X.T * self.y, 0).sum() + (self.reg / 2) * self.theta @ self.theta.T
    
    
    # Calculate the gradient of the objective value on the training examples. 
    def gradient(self):
        v = np.where((self.y * (self.X @ self.theta)) <= 1, 1, 0)
        tmp = (self.X.T * sp.diags((-self.y * v),0)).T.sum(axis=0) + self.reg * self.theta
        try:
            return tmp.A1
        except:
            return tmp
    
    # Train the support vector machine with the given parameters. 
    def train(self, niters=100, learning_rate=1, verbose=False):
        for t in range(niters):
            self.theta -= learning_rate * self.gradient()

    # Predict the class of each label in X. 
    def predict(self, X):
        return X @ self.theta



The reason why we want to use matrix operation (especially sparse matrix) here is that it is way much faster than mathematical calculation using for loops. The performance of our SVM classifier determines whether we can efficiently detect the spam emails. Let's try to time our training function to see how long does it takes train our email dataset.

In [37]:
# Train SVM on tfidf features using the labeled email dataset
emails, labels = get_data()
features, all_words = tfidf(emails)
svm = SVM(features, labels, 1e-4)
svm.train(verbose=True)

# Uncomment the line below to see how long training takes to run. 
# %timeit svm.train()

1.78 s ± 40 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Seems that fast enough to handle such a big dataset.

## Model Selection: Cross validation and Parameter Grid Search 
We might have noticed that there are parameters in the SVM learning algorithm that we chose somewhat arbitrarily: the regularization parameter and the learning rate

We can achieve perfect training accuracy with these random parameters. But since we have an enormous amount of features so it would be extremely easy to overfit to the data, so our model may not generalize well. 

We are now going to evaluate and select these parameters using cross validation and grid search.

## K-fold cross validation
There are many ways to evaluate the choice of parameters. One way is to perform k-fold cross validation, which operates as follows 

1. Split the data into k+1 randomly selected but uniformly sized pieces, and set aside one block for testing.
2. For each of the remaining k parts, train the model on k-1 parts and validate the model on the heldout part. 
3. This gives k results, and the average of these runs gives the final result.

The idea is that by holding out part of the dataset as validation data, we can train and measure our generalization ability. The training does not see the validation data at all, which is why it measures generalization. Randomizing the groups removes bias from ordering, and averaging over the groups reduces the variance. 

In this problem, we will use classification error rate as our result metric (so the fraction of times in which our model returns the wrong answer). Calculating this value via k-fold cross validation gives us a measure of how well our model generalizes to new data (lower error rate is better). 




## Grid search
Now, we have a means of evaluating our choice of parameters. We can now combine this with a grid search over parameters to determine the best combination. Given two lists of parameters, we compute the classification error using k-fold cross validation for each pair of parameters, and output the parameters that produces the best validation result. We are testing on learning rates [0.1, 1, 10] and regularization [0.01, 0.1, 1, 10] with 100 iterations and k=5. However, you can adjust the all these parameters based on practical need.

In [38]:

class ModelSelector:
    
    # Initialize the model selection with data and split into train/valid/test sets. Split the permutation into blocks 
    # and save the block indices as an attribute to the model. 
    def __init__(self, X, y, P, k, niters, SVM_impl):
        self.X = X
        self.y = y
        self.P = P
        self.k = k
        self.niters = niters
        self.svm = SVM_impl
        n = int(np.ceil(len(P)/(k+1)))
        blocks = [P[i:i+n] if i+n < len(P) else P[i:] for i in range(0, len(P), n)]
        self.blocks = blocks[:k]
        self.test_block = blocks[-1]

        
    # Evaluate the SVM using k-fold cross validation for the given parameters 
    def cross_validation(self, lr, reg):
        avg_err = []
        num = len(self.blocks)
        for i in range(num):
            train_rows = np.asarray([v for sub in range(num) if sub != i for v in self.blocks[sub]])
            valid_X = self.X[self.blocks[i],:]
            train_X = self.X[train_rows,:]
            valid_y = self.y[self.blocks[i]]
            train_y = self.y[train_rows]
            
            model = self.svm(train_X, train_y, reg)
            model.train(learning_rate=lr)
            pred_y = model.predict(valid_X)
            err = np.mean(pred_y*valid_y < 0)
            avg_err.append(err)
        
        return np.mean(avg_err)

    # Given two lists of parameters for learning rate and regularization parameter, perform a grid search using
    # k-wise cross validation to select the best parameters. 
    def grid_search(self, lrs, regs):
        import itertools
        best_lr = None
        best_reg = None
        best_err = None
        combs = list(itertools.product(lrs, regs))
        for comb in combs:
            lr, reg = comb[0], comb[1]
            curr_err = self.cross_validation(lr, reg)
            if best_err == None or curr_err < best_err:
                best_err = curr_err
                best_lr = lr
                best_reg = reg
        return (best_lr, best_reg)
    

    # Calculate the error rate of the test data given the rest of the data.      
    def test(self, lr, reg):
        train_rows = np.asarray([v for sub in range(len(self.blocks)) for v in self.blocks[sub]])
        train_X = self.X[train_rows,:]
        train_y = self.y[train_rows]
        test_X = self.X[self.test_block,:]
        test_y = self.y[self.test_block]
        model = self.svm(train_X, train_y, reg)
        model.train(learning_rate=lr)
        pred_y = model.predict(test_X)
        err = np.mean(pred_y*test_y < 0)
        return (err, model)
        

We are going to implement these functions on our email dataset to interpret how each method works. First let's import our data.

In [39]:
# Load the data and construct tfidf matrix
emails, labels = get_data()
features, all_words = tfidf(emails)
features

<10000x248087 sparse matrix of type '<class 'numpy.float64'>'
	with 2364991 stored elements in Compressed Sparse Row format>

We can see that the matrix is very sparse, as mentioned at the beginning of the note. Then we permutate the order of datapoints to ensure the randomness in cross validation. We use k=5, iterations =100, learning rate = 0.5 and regularization = 0.1 to test the functionality of cross validation.

In [40]:
# Assign random perturbation
P = np.random.permutation(features.shape[0])

# Initialize the model selector
MS = ModelSelector(features, labels, P, 5, 100, SVM)

# Caculate the classification error rate
cv_err = MS.cross_validation(0.5, 0.1)
assert(cv_err < 0.05)
cv_err

0.010197960407918417

Next we are going to perform grid search on learning rates and regularization. The grid search function will return the selected combination from learning rates [0.1, 1, 10] and regularization [0.01, 0.1, 1, 10] which gives us the lowest classification error rate.

In [41]:
lr, reg = MS.grid_search(np.array([0.001, 1., 10.]), np.array([0.0001, 0.1, 1.]))
lr,reg

(1.0, 0.1)

## Feature Compression 
While we can get decent results using an SVM and basic tf-idf features, there is one problem that we have to notice:
1. The number of features is extremely bloated and consumes a lot of space and computing power for a binary classification problem

So the above methodology would actually take a lot of time and memory to run on the full dataset. We save the tf-idf matrix for the entire training dataset (which is enormous), and then use that to generate features on new examples. 

One way to tackle this is to generate fewer, but effective, features. For example, instead of generating full tf-idf features for every single word in every email, we can instead try to focus on keywords that frequently only occur in spam email.

The implementation process is indicated as follow:

1. Split each document `d` into words using `d.split()`.
2. Determine the frequency with which words occur in spam and ham (not spam) emails. Count each appearance of each word.
3. Select words that only ever appear in spam and words that only ever appear in ham
4. From these words, return sets of those that appear at least `threshold` times.

In [42]:
# Given a list of emails and corresponding labels, return a list of frequent indicator words 
# for spam emails and for ham emails. 
def find_frequent_indicator_words(docs, y, threshold):
    words = [tuple(email.split()) for email in docs]
    labels = list(y)
    corr_list = list(zip(words, labels))
    spams = [w for i in corr_list if i[1] == -1.0 for w in i[0]]
    hams = [w for i in corr_list if i[1] == 1.0 for w in i[0]]
    spams_count = dict(Counter(spams))
    hams_count = dict(Counter(hams))
    spams_words, hams_words = set(spams_count.keys()), set(hams_count.keys())
    spams_only = [(w, spams_count[w]) for w in spams_words if w not in hams_words]
    hams_only = [(w, hams_count[w]) for w in hams_words if w not in spams_words]
    spams_freq = [item[0] for item in spams_only if item[1] >= threshold]
    hams_freq = [item[0] for item in hams_only if item[1] >= threshold]
    return (spams_freq, hams_freq)
    

We pass threshold as 50 to filter out those words that don't occur frequently. Let's see how great it can reduce the dimension of our feature matrix. 

In [43]:
emails, labels = get_data()
spam,ham = find_frequent_indicator_words(emails, labels, 50)
len(spam), len(ham)

(2422, 290)

Recall that our original feature matrix has more than 200,000 columns. This is a great reduction and save a lot of space and computation power.

## Efficient Spam Detection

Our final goal is to get at least 80% accuracy on spam email detection in an efficient manner. We will use the frequent indicator words implemented above and generate 2 features per email: the number of spam indicator words and the number of ham indicator words for a total of two features. As we metioned above, this is a huge dimensionality reduction.

Given the frequent indicator words, we will first generate 2 features per email for a given training dataset. Then, we will use ModelSelector to perform a grid search and train SVM classifier on the training dataset. Finally, we will evaluate the trained SVM on a separate test set, where we aim to achieve at least 80% accuracy.  

The function below generates the new email features based on the result of frequent indicator words. 

In [44]:
# Given a list of emails, create a matrix containing features for each email.
def email2features(emails, frequent_indicator_words):
    spams_freq, hams_freq = set(frequent_indicator_words[0]), set(frequent_indicator_words[1])
    spams_list, hams_list = [], []
    counter_list = [dict(Counter(email.split())) for email in emails]
    for item in counter_list:
        spams_dict, hams_dict = {}, {}
        for k in item:
            if k in spams_freq:
                spams_dict[k] = item[k]
            if k in hams_freq:
                hams_dict[k] = item[k]
        spams_list.append(spams_dict)
        hams_list.append(hams_dict)
    spams_list = [sum(item.values()) for item in spams_list]
    hams_list = [sum(item.values()) for item in hams_list]
    output = np.array(list(zip(spams_list, hams_list)))
    
    return output

    

The final detection function is implemented as follow.

In [46]:
def efficient_spam_detection_test(efficient_spam_detection_impl):
    emails, labels = get_data()
    emails_tr, labels_tr = load_data('Xtr_spam.txt.gz', 'ytr_spam.txt.gz')

    frequent_indicator_words = find_frequent_indicator_words(emails, labels, 50)    
    svm = efficient_spam_detection_impl(emails_tr, labels_tr, frequent_indicator_words)
    
    # Training error
    features = email2features(emails_tr, frequent_indicator_words)
    yp = svm.predict(features)
    print(np.mean(yp * labels_tr < 0))
    test.true(np.mean(yp * labels_tr < 0) <= 0.2)

# Given a training dataset of emails and labels, use the given frequent indicator words to
# generate features per email. Use ModelSelector to perform a grid search and train SVM
# on the training dataset
def efficient_spam_detection(emails, labels, frequent_indicator_words):
    
    features = email2features(emails, frequent_indicator_words)
    P = np.random.permutation(features.shape[0])
    MS = ModelSelector(features, labels, P, 5, 100, SVM)
    
    lr, reg = MS.grid_search(np.array([0.001, 1., 10.]), np.array([0.0001, 0.1, 1.]))
    svm = SVM(features, labels, reg)
    svm.train(niters=100, learning_rate=lr, verbose=False)
    return svm

emails, labels = get_data()
emails_tr, labels_tr = load_data('Xtr_spam.txt.gz', 'ytr_spam.txt.gz')

frequent_indicator_words = find_frequent_indicator_words(emails, labels, 50)    
svm = efficient_spam_detection(emails_tr, labels_tr, frequent_indicator_words)

# Training error
features = email2features(emails_tr, frequent_indicator_words)
yp = svm.predict(features)


np.mean(yp * labels_tr < 0)



0.131

## Conclusion
Finally we achieve our goal to reach accuracy of at least 80% in detecting spam emails. Notice that we are not predicting on our original dataset, instead we compress our features only selecting those words occurring frequently. It might seem to sacrifice the integrity of information in the dataset at first glance, but the actual functionality of those infrequent words can be limited. We can adjust the threshold to determine how we want to define "frequent". Also, there are lots of optimization and trade off options going on in this project, we have full flexibility over the learning rate, regularization and iterations to decide how we want our SVM classifier to work. One of the things that we can optimize is to do more pre-processing on our raw email dataset. What we have done is just separating each word with white space. More techniques (like regular expression) can be done to filtering the raw data based on practical need.

## References:

1. Scipy: https://www.scipy.org/
2. Numpy: https://numpy.org/